In [75]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.metrics import mean_squared_error

### import Data

In [76]:
train = pd.read_csv('../train.csv')
test = pd.read_csv('../test.csv')

In [77]:
print(train.shape)

(15035, 21)


In [78]:
train.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,0,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,1,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
2,2,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
3,3,20140627T000000,257500.0,3,2.25,1715,6819,2.0,0,0,...,7,1715,0,1995,0,98003,47.3097,-122.327,2238,6819
4,4,20150115T000000,291850.0,3,1.50,1060,9711,1.0,0,0,...,7,1060,0,1963,0,98198,47.4095,-122.315,1650,9711


In [79]:
test.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,15035,20141209T000000,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
1,15036,20141209T000000,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
2,15037,20140512T000000,4,4.50,5420,101930,1.0,0,0,3,11,3890,1530,2001,0,98053,47.6561,-122.005,4760,101930
3,15038,20150415T000000,3,1.00,1780,7470,1.0,0,0,3,7,1050,730,1960,0,98146,47.5123,-122.337,1780,8113
4,15039,20150312T000000,3,2.50,1890,6560,2.0,0,0,3,7,1890,0,2003,0,98038,47.3684,-122.031,2390,7570


In [80]:
train['date'] = train['date'].apply(lambda data: int(data[:8]))

In [81]:
train = train.drop('id', axis=1)

In [82]:
train['sqft_living'] = np.log1p(train['sqft_living'])
train['sqft_lot'] = np.log1p(train['sqft_lot'])
train['sqft_above'] = np.log1p(train['sqft_above'])
train['sqft_basement'] = np.log1p(train['sqft_basement'])
train['sqft_living15'] = np.log1p(train['sqft_living15'])
train['sqft_lot15'] = np.log1p(train['sqft_lot15'])

In [83]:
original_price = train['price']
train['price'] = np.log1p(train['price'])

In [84]:
y_label = train['price']
X_data = train.drop('price', axis=1)

### Make Tensor

In [85]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_data, y_label, train_size = 12000, test_size=3035, random_state=0)

In [86]:
X_train = torch.from_numpy(X_train.values).float()
y_train = torch.from_numpy(y_train.values).float()
X_test = torch.from_numpy(X_test.values).float()
y_test = torch.from_numpy(y_test.values).float()

In [87]:
print(X_train.shape)
print(y_train.shape)

torch.Size([12000, 19])
torch.Size([12000])


In [88]:
train = TensorDataset(X_train, y_train)
print(train[0])

(tensor([ 2.0150e+07,  3.0000e+00,  2.2500e+00,  7.9659e+00,  8.7149e+00,
         2.0000e+00,  0.0000e+00,  4.0000e+00,  4.0000e+00,  8.0000e+00,
         7.5606e+00,  6.8680e+00,  1.9830e+03,  0.0000e+00,  9.8116e+04,
         4.7588e+01, -1.2238e+02,  7.9659e+00,  8.5772e+00]), tensor(13.8662))


In [89]:
train_loader = DataLoader(train, batch_size=200, shuffle=True)

### Modeling

In [48]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(19, 10)
        self.fc2 = nn.Linear(10, 10)
        self.fc3 = nn.Linear(10, 5)
        self.fc4 = nn.Linear(5, 1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        return x

model = Net()

In [41]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10000):
    total_loss = 0
    for X_train, y_train in train_loader:
        X_train, y_train = Variable(X_train), Variable(y_train)
        optimizer.zero_grad()
        output = model(X_train)
        loss = criterion(output, y_train)
        loss.backward()
        optimizer.step()
        total_loss += float(loss.data)
        
    if (epoch+1) % 1000 == 0:
        print(epoch+1, total_loss)

1000 10230.647735595703
2000 10230.647659301758
3000 10230.647705078125
4000 10230.647659301758
5000 10230.647720336914
6000 10230.647674560547
7000 10230.647705078125
8000 10230.647720336914
9000 10230.647689819336
10000 10230.647659301758


In [42]:
X_test, y_test = Variable(X_test), Variable(y_test)

In [43]:
np.expm1(y_test)

tensor([379999.9375, 950000.0000, 580000.1250,  ..., 216500.0781,
        475000.1875, 629999.8750])

In [45]:
np.expm1(model(X_test).data.reshape(1,-1))[0]

tensor([0., 0., 0.,  ..., 0., 0., 0.])

In [47]:
np.sqrt(mean_squared_error(np.expm1(y_test), np.expm1(model(X_test).data.reshape(1, -1)[0])))

658650.56

In [49]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(19, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 10)
        self.fc4 = nn.Linear(10, 5)
        self.fc5 = nn.Linear(5, 1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x

model = Net()

In [50]:
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(1000):
    total_loss = 0
    for X_train, y_train in train_loader:
        X_train, y_train = Variable(X_train), Variable(y_train)
        optimizer.zero_grad()
        output = model(X_train)
        loss = criterion(output, y_train)
        loss.backward()
        optimizer.step()
        total_loss += float(loss.data)
        
    if (epoch+1) % 100 == 0:
        print(epoch+1, total_loss)

100 16.559765741229057
200 17.50287826359272
300 44.02097061276436
400 21.711525693535805
500 22.62486670911312
600 2907.9642066955566
700 265.1543679237366
800 16.517938315868378
900 16.518279433250427
1000 16.51884864270687


In [51]:
np.sqrt(mean_squared_error(np.expm1(y_test), np.expm1(model(X_test).data.reshape(1, -1)[0])))

378532.3

In [56]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(19, 25)
        self.fc2 = nn.Linear(25, 15)
        self.fc3 = nn.Linear(15, 10)
        self.fc4 = nn.Linear(10, 5)
        self.fc5 = nn.Linear(5, 1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x

model = Net()

In [58]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(3000):
    total_loss = 0
    for X_train, y_train in train_loader:
        X_train, y_train = Variable(X_train), Variable(y_train)
        optimizer.zero_grad()
        output = model(X_train)
        loss = criterion(output, y_train)
        loss.backward()
        optimizer.step()
        total_loss += float(loss.data)
        
    if (epoch+1) % 100 == 0:
        print(epoch+1, total_loss)

100 3747.3101077079773
200 67.08850800991058
300 83.04906523227692
400 17.712757036089897
500 16.522901996970177
600 16.540403842926025
700 16.53247508406639
800 16.582540839910507
900 16.5635873824358
1000 16.536832258105278
1100 16.54741172492504
1200 16.532775416970253
1300 16.55827985703945
1400 16.580873638391495
1500 16.545421972870827
1600 16.534170106053352
1700 16.548794150352478
1800 16.53825755417347
1900 16.5628664791584
2000 16.55444099009037
2100 16.520255908370018
2200 16.556858390569687
2300 16.547136887907982
2400 16.55953598022461
2500 16.575518563389778
2600 16.533291772007942
2700 16.552691653370857
2800 16.543199434876442
2900 16.522920712828636
3000 16.547366216778755


In [59]:
np.sqrt(mean_squared_error(np.expm1(y_test), np.expm1(model(X_test).data.reshape(1, -1)[0])))

383078.44

In [60]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(19, 30)
        self.fc2 = nn.Linear(30, 15)
        self.fc3 = nn.Linear(15, 10)
        self.fc4 = nn.Linear(10, 5)
        self.fc5 = nn.Linear(5, 1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x

model = Net()

In [61]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(3000):
    total_loss = 0
    for X_train, y_train in train_loader:
        X_train, y_train = Variable(X_train), Variable(y_train)
        optimizer.zero_grad()
        output = model(X_train)
        loss = criterion(output, y_train)
        loss.backward()
        optimizer.step()
        total_loss += float(loss.data)
        
    if (epoch+1) % 100 == 0:
        print(epoch+1, total_loss)

100 276.2766489982605
200 137.56556689739227
300 619.7788663804531
400 38.49732744693756
500 17.819962814450264
600 19.565630212426186
700 21.522492960095406
800 18.741317749023438
900 18.664339557290077
1000 19.2355148345232
1100 19.08906778693199
1200 17.076472103595734
1300 17.862764164805412
1400 18.68427187204361
1500 17.505357518792152
1600 18.099325001239777
1700 17.563391491770744
1800 17.85016320645809


KeyboardInterrupt: 

###  sqft 들 log

In [92]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(19, 40)
        self.fc2 = nn.Linear(40, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, 20)
        self.fc5 = nn.Linear(20, 10)
        self.fc6 = nn.Linear(10, 1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return x

model = Net()

In [93]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(3000):
    total_loss = 0
    for X_train, y_train in train_loader:
        X_train, y_train = Variable(X_train), Variable(y_train)
        optimizer.zero_grad()
        output = model(X_train)
        loss = criterion(output, y_train)
        loss.backward()
        optimizer.step()
        total_loss += float(loss.data)
        
    if (epoch+1) % 100 == 0:
        print(epoch+1, total_loss)

100 30.74306845664978
200 3188.236431121826
300 16.51838207244873
400 16.521115884184837
500 16.53012329339981
600 16.52767376601696
700 16.523153722286224
800 16.529236003756523
900 16.524161353707314
1000 16.527722090482712
1100 16.52660559117794
1200 16.525271847844124
1300 16.533432722091675
1400 16.52419039607048
1500 16.529597356915474
1600 16.539522141218185
1700 16.537590980529785
1800 16.530501663684845
1900 16.537803947925568
2000 16.52558383345604
2100 16.52595454454422
2200 16.52666413784027
2300 16.52531649172306
2400 16.532202944159508
2500 16.526219248771667
2600 16.530871644616127
2700 16.52795611321926
2800 16.523586079478264
2900 16.528484776616096
3000 16.524040549993515


In [94]:
np.sqrt(mean_squared_error(np.expm1(y_test), np.expm1(model(X_test).data.reshape(1, -1)[0])))

379960.5

In [103]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(19, 35)
        self.fc2 = nn.Linear(35, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, 15)
        self.fc5 = nn.Linear(15, 10)
        self.fc6 = nn.Linear(10, 1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return x

model = Net()


In [104]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(3000):
    total_loss = 0
    for X_train, y_train in train_loader:
        X_train, y_train = Variable(X_train), Variable(y_train)
        optimizer.zero_grad()
        output = model(X_train)
        loss = criterion(output, y_train)
        loss.backward()
        optimizer.step()
        total_loss += float(loss.data)
        
    if (epoch+1) % 100 == 0:
        print(epoch+1, total_loss)

100 9065.421951293945
200 28.009269148111343
300 16.519537538290024
400 16.544434174895287
500 16.528107330203056
600 16.53321009874344
700 16.530683934688568
800 16.522703632712364
900 16.526291966438293
1000 16.53238372504711
1100 16.55467027425766
1200 16.532466009259224
1300 16.530926540493965
1400 16.525581687688828
1500 16.52827464044094
1600 16.524475127458572
1700 16.52674624323845
1800 16.537703350186348
1900 16.529003649950027
2000 16.532664448022842
2100 16.524600625038147
2200 16.52873620390892
2300 16.52274239063263
2400 16.53641937673092
2500 16.524271219968796
2600 16.526543155312538
2700 16.523846358060837
2800 16.522902965545654
2900 16.5226189494133
3000 16.54248285293579


In [105]:
np.sqrt(mean_squared_error(np.expm1(y_test), np.expm1(model(X_test).data.reshape(1, -1)[0])))

377878.88